In [3]:
import os

from IPython import display
import matplotlib.pyplot as plot
import numpy as np
import pandas as pd
import csv
from functools import reduce

import tensorflow as tf
import tensorflow_hub as tfhub
import tensorflow_io as tfio
import tensorflow_model_optimization as tfmot

import params as yamnet_params
import yamnet as yamnet_model

In [4]:
# load in the yamnet model
params = yamnet_params.Params()
yamnet = yamnet_model.yamnet_frames_model(params)
yamnet.load_weights('yamnet.h5')
yamnet_classes = yamnet_model.class_names('yamnet_class_map.csv')

In [17]:
def apply_quantization(layer):
    print(layer.name)
    ls = [
        tf.keras.layers.Dense,
        tf.keras.layers.BatchNormalization,
        tf.keras.layers.Conv2D,
        # tf.nn.relu6,
        tf.keras.layers.DepthwiseConv2D,
    ]
    b = False
    for l in ls:
        # print(type(l))
        if isinstance(layer, l):
            b = True
    
    if b:
        print("annotating layer")
        return tfmot.quantization.keras.quantize_annotate_layer(layer)
    return layer

In [18]:
yamnet_annotated = tf.keras.models.clone_model(
    yamnet,
    clone_function=apply_quantization
)
yamnet_quant = tfmot.quantization.keras.quantize_apply(yamnet_annotated)


tf.signal.frame
tf.math.multiply
tf.compat.v1.shape
tf.__operators__.getitem
tf.math.subtract
tf.compat.v1.floor_div
tf.math.subtract_1
tf.math.subtract_2
tf.compat.v1.pad
tf.linalg.matmul
tf.linalg.matmul_1
tf.math.multiply_1
tf.math.multiply_2
tf.math.add
tf.math.sqrt
tf.linalg.matmul_2
tf.__operators__.add
tf.math.log
tf.signal.frame_1
reshape
layer1/conv
annotating layer
layer1/conv/bn
annotating layer
tf.nn.relu6
layer2/depthwise_conv
annotating layer
layer2/depthwise_conv/bn
annotating layer
tf.nn.relu6_1
layer2/pointwise_conv
annotating layer
layer2/pointwise_conv/bn
annotating layer
tf.nn.relu6_2
layer3/depthwise_conv
annotating layer
layer3/depthwise_conv/bn
annotating layer
tf.nn.relu6_3
layer3/pointwise_conv
annotating layer
layer3/pointwise_conv/bn
annotating layer
tf.nn.relu6_4
layer4/depthwise_conv
annotating layer
layer4/depthwise_conv/bn
annotating layer
tf.nn.relu6_5
layer4/pointwise_conv
annotating layer
layer4/pointwise_conv/bn
annotating layer
tf.nn.relu6_6
layer5/d

In [19]:
# convert to tflite
converter = tf.lite.TFLiteConverter.from_keras_model(yamnet_quant)
# converter = tf.lite.TFLiteConverter.from_keras_model(yamnet)
tflite_model = converter.convert()
# save to file called me/yamnet.tflite
# with open('models/3/me/features.tflite', 'wb') as f:
with open('/models/3/me/yamnet_quant.tflite', 'wb') as f:
    f.write(tflite_model)

FileNotFoundError: [Errno 2] No such file or directory: 'models/3/me/yamnet_quant.tflite'

In [22]:
with open('models/3/yamnet_quant.tflite', 'wb') as f:
    f.write(tflite_model)

In [23]:
yamnet_quant.summary()

Model: "yamnet_frames"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
waveform_binary 0 (InputLayer)  [(15600,)]           0                                            
__________________________________________________________________________________________________
tf.signal.frame (TFOpLambda)    (96, 400)            0           waveform_binary 0[0][0]          
__________________________________________________________________________________________________
tf.math.multiply (TFOpLambda)   (96, 400)            0           tf.signal.frame[1][0]            
__________________________________________________________________________________________________
tf.compat.v1.shape (TFOpLambda) (2,)                 0           tf.math.multiply[1][0]           
______________________________________________________________________________________